<a href="https://colab.research.google.com/github/archarbar/Fashion-MNIST-Challenge/blob/master/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files
token = files.upload()

In [0]:
import os
os.chdir(r'/content/drive/My Drive/MAIS 202 Assignments/Assignment 4')

In [0]:
import os
os.chdir(r'/content/drive/My Drive/')

In [0]:
!mkdir /root/.kaggle/
!cp kaggle.json /root/.kaggle/ 
!kaggle competitions download -c winter2020-mais-202

cp: cannot stat 'kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
!unzip -o 'train_images.npy.zip' 
!unzip -o 'test_images.npy.zip'

unzip:  cannot find or open train_images.npy.zip, train_images.npy.zip.zip or train_images.npy.zip.ZIP.
unzip:  cannot find or open test_images.npy.zip, test_images.npy.zip.zip or test_images.npy.zip.ZIP.


In [0]:
# !pip install numpy

import keras
from keras import optimizers
from keras.datasets import mnist
from keras.applications import VGG16, VGG19, InceptionResNetV2, Xception, NASNetLarge
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.utils import shuffle

import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import csv
import glob
import random
import math

import tensorflow as tf


Using TensorFlow backend.


In [0]:
!tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt

TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 0 files
Detected 0 issues that require attention
--------------------------------------------------------------------------------


Make sure to read the detailed log 'report.txt'



In [0]:
!pwd

/content/drive/My Drive


# Preprocessing

In [0]:
train_images = np.load('train_images.npy')
test_images = np.load('test_images.npy')

#reshape into 2 dimensions
X_train = train_images.reshape(len(train_images),-1)
X_train = X_train

X_test = test_images.reshape(len(test_images),-1)
X_test = X_test
# print(train_images)

# print(X_train)
# print(len(train_images))

y_train = pd.read_csv('train_labels.csv')
y_train = y_train["label"]

#shuffle dataset
X_train, y_train = shuffle(X_train, y_train)

In [0]:
y_train[:10]

24965    4
15158    3
3565     3
7997     6
5003     0
1478     8
2802     0
31605    2
36691    4
15853    2
Name: label, dtype: int64

In [0]:
y = y_train.to_numpy()

In [0]:
y_train.shape

(50000,)

In [0]:
X_train.shape

(40000, 784)

In [0]:
y[:10]

array([4, 3, 3, 6, 0, 8, 0, 2, 4, 2])

In [0]:
y.shape

(50000,)

In [0]:
X_train.shape

(50000, 784)

### Random Forest

In [0]:
rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)
print('Training accuracy: ', rfc.score(X_train, y_train))

Training accuracy:  1.0


In [0]:
y = rfc.predict(X_test)
print(y)

[9 5 2 ... 3 8 6]


In [0]:
with open('submission.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['ID', 'label'])

    for c, row in enumerate(y):
      writer.writerow((c, row))

# CNN

In [0]:
import os
os.chdir(r'/content/drive/My Drive/McGill/Winter 2020/MAIS 202 Bootcamp/Assignment/Assignment 4')

In [0]:
X_train = np.load('train_images.npy')
X_test = np.load('test_images.npy')

y_train = pd.read_csv('train_labels.csv')
y_train = y_train["label"]
y_train = y_train.to_numpy()

In [0]:
X_train = X_train.reshape(50000, 28, 28, 1)
X_test = X_test.reshape(20000, 28, 28, 1)

In [0]:
X_train = X_train / 255
X_test = X_test / 255

In [0]:
from google.colab.patches import cv2_imshow

cv2_imshow(X_train[3])
y_train[3]

1

In [0]:
X_train.shape

(50000, 28, 28, 1)

In [0]:
# create model
model = Sequential()

# add convolutional layer
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28, 28, 1)))

model.add(Conv2D(64, (3,3), activation='relu'))

# add pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# add dropout layer (prevent neural network from overfitting)
model.add(Dropout(0.25))

# add flatten layer to flatten into a 1D array
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.50))

# add dense layer to predict output
model.add(Dense(10, activation='softmax'))

In [0]:
activation = keras.layers.LeakyReLU(alpha=0.3)

In [0]:
# create model 2
model = Sequential()

# add convolutional layer
model.add(Conv2D(32, kernel_size=(3,3), use_bias=True, activation='relu', input_shape=(28, 28, 1)))

model.add(Conv2D(64, (3,3), activation='relu'))

# add pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# add dropout layer (prevent neural network from overfitting)
#model.add(Dropout(0.25))

model.add(Conv2D(32, (3,3), use_bias=True, activation='relu'))
model.add(Conv2D(16, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(16, (3,3), padding='same', activation=activation))
model.add(Conv2D(32, (3,3), padding='same', activation=activation))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

# add flatten layer to flatten into a 1D array
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.50))

# model.add(Dense(64, activation='softmax'))

# add dense layer to predict output
model.add(Dense(10, activation='softmax'))

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [0]:
optimizer = optimizers.Adadelta()

In [0]:
# compile model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_153 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_154 (Conv2D)          (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_65 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_155 (Conv2D)          (None, 10, 10, 32)        18464     
_________________________________________________________________
conv2d_156 (Conv2D)          (None, 10, 10, 16)        4624      
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
conv2d_157 (Conv2D)          (None, 5, 5, 16)        

In [0]:
# Assume that you have 12GB of GPU memory and want to allocate ~4GB:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [0]:
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2, shuffle=True)

Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 8s 207us/step - loss: 0.2836 - acc: 0.8981 - val_loss: 0.4344 - val_acc: 0.8503
Epoch 2/30
40000/40000 [==============================] - 8s 209us/step - loss: 0.2776 - acc: 0.8994 - val_loss: 0.4766 - val_acc: 0.8605
Epoch 3/30
40000/40000 [==============================] - 8s 209us/step - loss: 0.2730 - acc: 0.9014 - val_loss: 0.4607 - val_acc: 0.8513
Epoch 4/30
40000/40000 [==============================] - 8s 210us/step - loss: 0.2685 - acc: 0.9038 - val_loss: 0.4676 - val_acc: 0.8587
Epoch 5/30
18432/40000 [============>.................] - ETA: 4s - loss: 0.2665 - acc: 0.9029

KeyboardInterrupt: ignored

In [0]:
model.save_weights('CNN2.h5')